仓库管理系统
https://github.com/r135792uuuu/Warehouse-Management-System.git

In [21]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox

# Load databases
inventory_db_path = 'E:\\Program\\WarehouseManageSystem\\database\\database1.xlsx'
borrow_return_db_path = 'E:\\Program\\WarehouseManageSystem\\database\\database2.xlsx'

try:
    inventory_df = pd.read_excel(inventory_db_path, engine='openpyxl')
    borrow_return_df = pd.read_excel(borrow_return_db_path, engine='openpyxl')
except FileNotFoundError:
    messagebox.showerror("错误", "没找到数据库，检查E盘数据是不是被删掉了.")
    exit()

# Functions
def view_inventory():
    inventory_window = tk.Toplevel(root)
    inventory_window.title("Inventory")
    text = tk.Text(inventory_window)
    text.pack()
    for index, row in inventory_df.iterrows():
        text.insert(tk.END, f"{row['大类名称']} - {row['小类名称']}: {row['数量']} 放在 {row['存放位置']}\n")

def view_borrow_return():
    borrow_return_window = tk.Toplevel(root)
    borrow_return_window.title("Borrow/Return Records")
    text = tk.Text(borrow_return_window)
    text.pack()
    for index, row in borrow_return_df.iterrows():
        text.insert(tk.END, f"人员: {row['保管人员']}, {row['借出物品大类名称']} - {row['借出物品小类名称']}: {row['借出物品数量']} ({row['物品状态']})\n")


def add_inventory_item():
    try:
        category = category_entry.get()
        subcategory = subcategory_entry.get()
        quantity = int(quantity_entry.get())
        location = f"{location_choice.get()}-{cabinet_number.get()}-{detailed_description.get()}"
        remark = remark_entry.get()

        if quantity <= 0:
            raise ValueError("输入正数！fk you！.")

        new_item = {'大类名称': category, '小类名称': subcategory, '数量': quantity, '存放位置': location, '备注': remark}
        global inventory_df
        inventory_df = inventory_df.append(new_item, ignore_index=True)
        inventory_df.to_excel(inventory_db_path, index=False, engine='openpyxl')
        messagebox.showinfo("成功", "添加物品成功!")
    except ValueError as e:
        messagebox.showerror("Error", f"Invalid input: {e}")

def update_databases():
    try:
        borrower = borrower_entry.get()
        category = category_entry2.get()
        subcategory = subcategory_entry2.get()
        quantity = int(quantity_entry2.get())
        status = status_entry.get()

        if quantity <= 0:
            raise ValueError("不要乱写负数！fk你！.")

        # Update borrow/return database
        new_borrow_entry = {
            '借出物品大类名称': category,
            '借出物品小类名称': subcategory,
            '借出物品数量': quantity,
            '保管人员': borrower,
            '物品状态': status,
            '备注': ''
        }
        global borrow_return_df
        borrow_return_df = borrow_return_df.append(new_borrow_entry, ignore_index=True)
        borrow_return_df.to_excel(borrow_return_db_path, index=False, engine='openpyxl')

        # Update inventory database
        inventory_index = inventory_df[(inventory_df['大类名称'] == category) & (inventory_df['小类名称'] == subcategory)].index

        if not inventory_index.empty:
            if status == '借出':
                inventory_df.at[inventory_index[0], '数量'] -= quantity
            elif status in ['归还', '采购']:
                inventory_df.at[inventory_index[0], '数量'] += quantity
            elif status in ['交付', '损坏']:
                inventory_df.at[inventory_index[0], '数量'] -= quantity

        inventory_df.to_excel(inventory_db_path, index=False, engine='openpyxl')
        messagebox.showinfo("成功", "数据库更新成功！")
    except ValueError as e:
        messagebox.showerror("Error", f"Invalid input: {e}")

def search_borrower_items():
    borrower_name = search_entry.get()
    search_window = tk.Toplevel(root)
    search_window.title(f"Items borrowed by {borrower_name}")
    text = tk.Text(search_window)
    text.pack()
    
    # Filter records for the specified borrower
    borrower_records = borrow_return_df[borrow_return_df['保管人员'] == borrower_name]
    
    if borrower_records.empty:
        text.insert(tk.END, f"No records found for {borrower_name}.\n")
    else:
        current_count = {}
        delivered_count = {}
        damaged_count = {}
        
        for index, row in borrower_records.iterrows():
            subcategory = row['借出物品小类名称']
            quantity = row['借出物品数量']
            status = row['物品状态']
            
            if status == '借出':
                current_count[subcategory] = current_count.get(subcategory, 0) + quantity
            elif status == '归还':
                current_count[subcategory] = current_count.get(subcategory, 0) - quantity
            elif status == '交付':
                delivered_count[subcategory] = delivered_count.get(subcategory, 0) + quantity
            elif status == '损坏':
                damaged_count[subcategory] = damaged_count.get(subcategory, 0) + quantity
        
        # Display results
        current_items = ", ".join([f"{count} 个 {name}" for name, count in current_count.items()])
        delivered_items = ", ".join([f"{name} 数量：{count}" for name, count in delivered_count.items()])
        damaged_items = ", ".join([f"{name}个数：{count}" for name, count in damaged_count.items()])
        
        text.insert(tk.END, f"当前名下还有：{current_items}。\n")
        text.insert(tk.END, f"交付{delivered_items}。\n")
        text.insert(tk.END, f"损坏{damaged_items}。\n")


# GUI setup
root = tk.Tk()
root.title("仓库管理系统-v0.1-yhw最帅版")
view_button = tk.Button(root, text="----------------------------------->点击按钮，守护最棒的kd大将军<----------------------------------")
view_button.pack()


# Frame for Inventory Management
inventory_frame = tk.Frame(root)
inventory_frame.pack(pady=10)

tk.Label(inventory_frame, text="大类别名称（NX，飞控...）").grid(row=0, column=0)
category_entry = tk.Entry(inventory_frame)
category_entry.grid(row=0, column=1)

tk.Label(inventory_frame, text="子类别名称（8g核心板，底板，nxt v1, ...）").grid(row=1, column=0)
subcategory_entry = tk.Entry(inventory_frame)
subcategory_entry.grid(row=1, column=1)

tk.Label(inventory_frame, text="数量").grid(row=2, column=0)
quantity_entry = tk.Entry(inventory_frame)
quantity_entry.grid(row=2, column=1)

# Location selection using dropdowns
tk.Label(inventory_frame, text="存放位置（门号-柜子号-细分描述）").grid(row=3, column=0)
location_choice = tk.StringVar(value="627")
location_menu = tk.OptionMenu(inventory_frame, location_choice, "627", "629")
location_menu.grid(row=3, column=1)

cabinet_number = tk.StringVar()
cabinet_entry = tk.Entry(inventory_frame, textvariable=cabinet_number)
cabinet_entry.grid(row=3, column=2)

detailed_description = tk.StringVar()
description_entry = tk.Entry(inventory_frame, textvariable=detailed_description)
description_entry.grid(row=3, column=3)

tk.Label(inventory_frame, text="备注").grid(row=4, column=0)
remark_entry = tk.Entry(inventory_frame)
remark_entry.grid(row=4, column=1)

add_button = tk.Button(inventory_frame, text="添加物品", command=add_inventory_item)
add_button.grid(row=5, columnspan=2, pady=5)

# Frame for Borrow/Return Management
borrow_return_frame = tk.Frame(root)
borrow_return_frame.pack(pady=10)

tk.Label(borrow_return_frame, text="借还人员").grid(row=0, column=0)
borrower_entry = tk.Entry(borrow_return_frame)
borrower_entry.grid(row=0, column=1)

tk.Label(borrow_return_frame, text="借还状态（借出，归还，交付，采购, 损坏）").grid(row=1, column=0)
status_entry = tk.Entry(borrow_return_frame)
status_entry.grid(row=1, column=1)

tk.Label(borrow_return_frame, text="大类别名称").grid(row=2, column=0)
category_entry2 = tk.Entry(borrow_return_frame)
category_entry2.grid(row=2, column=1)

tk.Label(borrow_return_frame, text="子类别名称").grid(row=3, column=0)
subcategory_entry2 = tk.Entry(borrow_return_frame)
subcategory_entry2.grid(row=3, column=1)

tk.Label(borrow_return_frame, text="数量").grid(row=4, column=0)
quantity_entry2 = tk.Entry(borrow_return_frame)
quantity_entry2.grid(row=4, column=1)

update_button = tk.Button(borrow_return_frame, text="点击更新数据库", command=update_databases)
update_button.grid(row=5, columnspan=2, pady=5)

# Frame for Search Functionality
search_frame = tk.Frame(root)
search_frame.pack(pady=10)

tk.Label(search_frame, text="查找你名下的财产").grid(row=0, column=0)
search_entry = tk.Entry(search_frame)
search_entry.grid(row=0, column=1)

search_button = tk.Button(search_frame, text="开始查找吧~", command=search_borrower_items)
search_button.grid(row=1, columnspan=2, pady=5)

# Frame for Viewing Records
view_frame = tk.Frame(root)
view_frame.pack(pady=10)

view_button = tk.Button(view_frame, text="查看仓库", command=view_inventory)
view_button.grid(row=0, column=0, padx=5)

view_borrow_return_button = tk.Button(view_frame, text="查看借还记录", command=view_borrow_return)
view_borrow_return_button.grid(row=0, column=1, padx=5)

root.mainloop()